# Requisições à API dos Circulares da USP

A API dos circulares da USP, disponível em https://uspdigital.usp.br/mobile/servicos/sptrans/posicoes, indica os veículos que estão trafegando no momento na universidade. Tais dados são úteis para o rastreamento dos ônibus e a coleta de dados geral a respeito dos tempos de viagem, por exemplo.

In [46]:
import requests
import json
import datetime
import time

In [59]:
class Bus:
    def __init__(self, bus_id, bus_line, last_register, position):
        self.bus_id = bus_id
        self.bus_line = bus_line
        self.last_register = last_register
        self.position = position

In [73]:
class State:
    def __init__(self, url):
        self.bus_list = []
        self.data_register = datetime.datetime.now()
        
        dict_api_states = self.request_api_state(url)
        self.create_vehicles(dict_api_states)
        
    def request_api_state(self, url):
        """ A função tem o objetivo de realizar uma requisição a API de endereço url, 
        retornando um dicionário com o estado atual dos ônibus na universidade """
        
        try:
            s_dict = requests.get(url).text 
            dict = json.loads(s_dict)
        except Exception as e:
            dict = []
            print(e)
        
        return dict
    
    def create_vehicles(self, dict):
        for bus_line in dict:
            for bus in bus_line["vs"]:
                bus_position = (bus["px"], bus["py"])
                obj_bus = Bus(bus["p"], bus_line["l"], bus["ta"], bus_position)
                
                self.bus_list.append(obj_bus)
    
    def get_vehicles(self):
        return self.bus_list
    
    def dump_list(self):
        i = 0
        str = '{"data_register": "' + self.data_register.strftime("%Y-%m-%dT%H:%M:%SZ") + '", '
        str += '"buses": ['
        for bus in self.bus_list:
            str += json.dumps(vars(bus))
            if i < len(self.bus_list) - 1:
                str += ','
            i += 1
            
        return str + ']}'

## Registro de dados de estados

Um estado da API, representado por uma lista de ônibus, é bastante informativo no quesito de dados necessários à análise de questões referentes aos veículos. O registro de estado pode, então, ser realizado em um arquivo JSON em um dado período de tempo.

In [68]:
def data_register(duration_time, x, url):
    """ Dado um tempo duration_time em segundos, registra os dados de estados em um arquivo JSON
    a cada x segundos """
    
    now = datetime.datetime.now()
    duration = datetime.timedelta(seconds=duration_time)
    endtime = now + duration
    
    while datetime.datetime.now() <= endtime:
        curr_state = State(url)
        with open('data/states.json', 'a') as f:
            f.write(curr_state.dump_list() + '\n')
        time.sleep(x)

## Leitura de dados escritos

A partir do registro de dados realizado pela função data_register, precisamos ler o histórico do arquivo para recuperar a lista de estados. Isso é feito pela função data_read, que, dado o nome do arquivo, retorna a lista que representa o conjunto de todos os estados registrados naquele arquivo.

In [36]:
def data_read(file):
    """ Retorna uma lista de estados dado um arquivo file que os contenha no formato definido na função
    data_register """
    
    states = []
    with open(file, 'r') as f:
        for line in f.readlines():
            state = json.loads(line)
            states.append(state)
    return states